In [3]:
!pip install seaborn matplotlib imblearn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.0/8.0 MB 5.5 MB/s eta 0:00:02
   ----------- ---------------------------- 2.4/8.0 MB 6.3 MB/s eta 0:00:01
   ---------------------- ----------------- 4.5/8.0 MB 7.1 MB/s eta 0:00:01
   ----------------------------------- ---- 7.1/8.0 MB 8.6 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 8.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------- ----------- 1.6/2.2 MB 7.2 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.6 MB ? eta -:--:--
   ----------- ---------------------------- 0.8/2.6 MB 2.2 MB/s eta 0:00:

In [4]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle

In [7]:
# Load the dataset
df = pd.read_csv('C:\\Users\\HP\\Downloads\\Customer-Churn-Records.csv')

# Display basic information about the dataset
print(df.info())
print(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   RowNumber           10000 non-null  int64  
 1   CustomerId          10000 non-null  int64  
 2   Surname             10000 non-null  object 
 3   CreditScore         10000 non-null  int64  
 4   Geography           10000 non-null  object 
 5   Gender              10000 non-null  object 
 6   Age                 10000 non-null  int64  
 7   Tenure              10000 non-null  int64  
 8   Balance             10000 non-null  float64
 9   NumOfProducts       10000 non-null  int64  
 10  HasCrCard           10000 non-null  int64  
 11  IsActiveMember      10000 non-null  int64  
 12  EstimatedSalary     10000 non-null  float64
 13  Exited              10000 non-null  int64  
 14  Complain            10000 non-null  int64  
 15  Satisfaction Score  10000 non-null  int64  
 16  Card 

In [8]:
# Define numerical and categorical columns for reference
numerical_columns = ['RowNumber', 'CustomerId', 'Surname', 'CreditScore',
                     'Age', 'Balance', 'EstimatedSalary', 'Point Earned']

In [9]:
# Map Card Type to numerical values
def map_card_tiers(df, column_name):
    card_map = {'SILVER': 1, 'GOLD': 2, 'DIAMOND': 3, 'PLATINUM': 4}
    df['Card_Tier'] = df[column_name].map(card_map)
    return df

df = map_card_tiers(df, 'Card Type')

In [10]:
# Clean column names by replacing spaces with underscores
df.columns = df.columns.str.replace(' ', '_')

# Remove unnecessary columns
df = df.drop(['RowNumber', 'CustomerId', 'Surname', 'Card_Type'], axis=1)

In [11]:
# Handle categorical variables using one-hot encoding
df_dummies = pd.get_dummies(df[['Geography', 'Gender']], dtype=int)
df_clean = pd.concat([df, df_dummies], axis=1)
df_clean = df_clean.drop(['Geography', 'Gender'], axis=1)

In [12]:
# Rename the target variable for clarity
df_clean['Churn'] = df_clean['Exited']
df_clean = df_clean.drop(['Exited'], axis=1)
df_clean.head(2)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Complain,Satisfaction_Score,Point_Earned,Card_Tier,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,Churn
0,619,42,2,0.00,1,1,1,101348.88,1,2,464,3,1,0,0,1,0,1
1,608,41,1,83807.86,1,0,1,112542.58,1,3,456,3,0,0,1,1,0,0


In [13]:
# Oversample the minority class using SMOTE
smote = SMOTE(random_state=42)
X = df_clean.drop(columns='Churn')
y = df_clean['Churn']
X_resampled, y_resampled = smote.fit_resample(X, y)

In [14]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Initialize the Random Forest model
rfc = RandomForestClassifier(random_state=42)

In [15]:
# Define hyperparameter grid for optimization
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [16]:
# Perform hyperparameter tuning using GridSearchCV
grid_search = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

In [17]:
# Train the model with the best parameters
print("Starting GridSearchCV...")
grid_search.fit(X_train, y_train)

Starting GridSearchCV...
Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150]},
             scoring='accuracy', verbose=2)

In [18]:
# Display the best parameters
print(f"Best Parameters: {grid_search.best_params_}")

Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}


In [19]:
# Evaluate the model using the test set
best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test)

In [20]:
# Calculate accuracy and display classification metrics
accuracy = accuracy_score(y_test, y_test_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test, y_test_pred))

Accuracy: 1.00
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1651
           1       1.00      1.00      1.00      1534

    accuracy                           1.00      3185
   macro avg       1.00      1.00      1.00      3185
weighted avg       1.00      1.00      1.00      3185



In [21]:
# Save the trained model for future use
with open('customer_churn_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

print("Model saved successfully!")


Model saved successfully!
